# Final Dataframe Combination Script

Author(s): Krutika Ingale <br>
Project Manager: Jaren Haber, PhD Candidate <br>
Contact: jhaber@berkeley.edu

Institution: University of California, Berkeley <br>
Program: Undergraduate Research Apprentice Program (URAP) <br>

Date created: <br>
Last modified: 11/12/18

Description: Notebook to combine our around 11 dataframes of overlap removed webtext and cmo_webtext

In [6]:
#import necessary packages 
import sqlite3
import csv
import pandas as pd
import os
import re

import math
from tqdm import tqdm
import multiprocessing as mp
import time
import sched
import ast
import collections

import numpy as np
from itertools import groupby

folder_prefix = '/home/jovyan/work/'

In [3]:
#read in original data frame which we needed to remove overlaps from
original_df = pd.read_pickle(folder_prefix + 'nowdata/backups/charters_full_2015_250_new_counts.pkl')

In [3]:
original_df

,CMO_NAME,CMO_MEMSUM,SCH_NAME,CMO_STATE,CMO_SCHNUM,CMO_URL,CMO_NUMSTATES,CMO_ALLSTATES,CMO_SECTOR,CMO_NUMSTUDENTS_CREDO17,...,STR,WEBTEXT_METHOD,ess_strength_y,prog_strength_y,PROG_COUNT,RIT_COUNT,ESS_COUNT,ESS_STR,PROG_STR,RIT_STR
0,AAEC,1320.0,ARIZONA AGRIBUSINESS & EQUINE CENTER - ESTRELLA,AZ,6.0,https://www.aaechighschools.com/,1.0,AZ,NaN,NaN,...,-480.000000,0,0.087500,0.037500,1,7,0,-6.000000,-2.977266,-2.132168
1,AAEC,1320.0,ARIZONA AGRIBUSINESS & EQUINE CENTER - ESTRELLA,AZ,6.0,https://www.aaechighschools.com/,1.0,AZ,NaN,NaN,...,-480.000000,0,0.087500,0.037500,1,7,0,-6.000000,-2.977266,-2.132168
2,Arizona Agribusiness & Equine Center,1320.0,ARIZONA AGRIBUSINESS & EQUINE CENTER - ESTRELLA,AZ,6.0,https://www.aaechighschools.com/,1.0,AZ,CMO,1220.0,...,-480.000000,0,0.087500,0.037500,1,7,0,-6.000000,-2.977266,-2.132168
3,Arizona Agribusiness & Equine Center,1320.0,ARIZONA AGRIBUSINESS & EQUINE CENTER - ESTRELLA,AZ,6.0,https://www.aaechighschools.com/,1.0,AZ,CMO,1220.0,...,-480.000000,0,0.087500,0.037500,1,7,0,-6.000000,-2.977266,-2.132168
4,AAEC,1320.0,AAEC - SMCC CAMPUS,AZ,6.0,https://www.aaechighschools.com/,1.0,AZ,NaN,NaN,...,-434.000000,0,0.092505,0.097047,1,8,0,-6.000000,-2.941014,-2.037426
5,AAEC,1320.0,AAEC - SMCC CAMPUS,AZ,6.0,https://www.aaechighschools.com/,1.0,AZ,NaN,NaN,...,-434.000000,0,0.092505,0.097047,1,8,0,-6.000000,-2.941014,-2.037426
6,Arizona Agribusiness & Equine Center,1320.0,AAEC - SMCC CAMPUS,AZ,6.0,https://www.aaechighschools.com/,1.0,AZ,CMO,1220.0,...,-434.000000,0,0.092505,0.097047,1,8,0,-6.000000,-2.941014,-2.037426
7,Arizona Agribusiness & Equine Center,1320.0,AAEC - SMCC CAMPUS,AZ,6.0,https://www.aaechighschools.com/,1.0,AZ,CMO,1220.0,...,-434.000000,0,0.092505,0.097047,1,8,0,-6.000000,-2.941014,-2.037426
8,AAEC,1320.0,AAEC - PARADISE VALLEY,AZ,6.0,https://www.aaechighschools.com/,1.0,AZ,NaN,NaN,...,-406.000000,0,0.083333,0.020833,0,1,0,-6.000000,-6.000000,-2.664642
9,AAEC,1320.0,AAEC - PARADISE VALLEY,AZ,6.0,https://www.aaechighschools.com/,1.0,AZ,NaN,NaN,...,-406.000000,0,0.083333,0.020833,0,1,0,-6.000000,-6.000000,-2.664642


### Load and combine parsed chunks into one big DF

Read in 11 dataframes with parsed text

In [4]:
parsed_df_1 = pd.read_csv(folder_prefix + 'nowdata/parsing/parsed_df_1.tar.gz', compression='gzip', header=0, sep='\t', quotechar='"')


/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (23,31,115,140,143,146,149,152,155,158,163,168,173,198) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
parsed_df_1.rename(columns={'parsed_df_1.csv': 'CMO_NAME'}, inplace=True) # large file!
#save the stopwords 


In [ ]:
parsed_df_2 = pd.read_csv(folder_prefix + "nowdata/parsing/parsed_df_2.csv", sep="\t", low_memory=False, encoding="utf-8")

In [ ]:
parsed_df_3 = pd.read_csv(folder_prefix + "nowdata/parsing/parsed_df_3.csv", sep="\t", low_memory=False, encoding="utf-8")
parsed_df_4 = pd.read_csv(folder_prefix + "nowdata/parsing/parsed_df_4.csv", sep="\t", low_memory=False, encoding="utf-8")

parsed_df_5 = pd.read_csv(folder_prefix + "nowdata/parsing/parsed_df_5.csv", sep="\t", low_memory=False, encoding="utf-8")

parsed_df_6 = pd.read_csv(folder_prefix + "nowdata/parsing/parsed_df_6.csv", sep="\t", low_memory=False, encoding="utf-8")

In [6]:
parsed_df_7 = pd.read_csv(folder_prefix + "nowdata/parsing/parsed_df_7.csv", sep="\t", low_memory=False, encoding="utf-8") # large file

In [12]:
parsed_df_8 = pd.read_csv(folder_prefix + "nowdata/parsing/parsed_df_8.csv", sep="\t", low_memory=False, encoding="utf-8")

In [13]:
parsed_df_9 = pd.read_csv(folder_prefix + "nowdata/parsing/parsed_df_9.csv", sep="\t", low_memory=False, encoding="utf-8")

In [14]:
parsed_df_10 = pd.read_csv(folder_prefix + "nowdata/parsing/parsed_df_10.csv", sep="\t", low_memory=False, encoding="utf-8")

In [15]:
parsed_df_11 = pd.read_csv(folder_prefix + "nowdata/parsing/parsed_df_11.csv", sep="\t", low_memory=False, encoding="utf-8")

In order to merge these DFs, we need to make sure they have the same columns. However, there's a problem: we switched source files 3/4 of the way into the process, resulting in some columns in the new data that weren't in the old data. So next we check which columns are in which DFs, and add the missing ones to all. 

In [21]:
# Compare first DF to 10th DF, identify different columns:
diff_cols = []
for i in list(parsed_df_10.columns):
    if i not in list(parsed_df_1.columns):
        diff_cols.append(i)
        print(i)

In [17]:
new_cols = ["WEBTEXT_METHOD", "REPLACED", "CMO_REPLACED", "PROG_COUNT", "RIT_COUNT","ESS_COUNT", "ESS_STR","PROG_STR","RIT_STR", "NUMWORDS"]

Add new columns to the dataframes which don't have them

In [18]:
for col_name in new_cols:
    parsed_df_1[col_name] = 0
    parsed_df_2[col_name] = 0
    parsed_df_3[col_name] = 0
    parsed_df_4[col_name] = 0
    parsed_df_5[col_name] = 0

In [19]:
other_cols = ["PROG_COUNT", "RIT_COUNT","ESS_COUNT","ESS_STR","PROG_STR","RIT_STR","NUMWORDS"]

In [20]:
for col_name in other_cols:
    parsed_df_6[col_name] = 0
    parsed_df_7[col_name] = 0

In [23]:
li_dfs = [parsed_df_1, parsed_df_2, parsed_df_3, parsed_df_4, parsed_df_5, parsed_df_6, parsed_df_7, parsed_df_8, parsed_df_9, parsed_df_10, parsed_df_11]


Concatenate the 11 dfs together

In [24]:
full_data = pd.concat(li_dfs, ignore_index = True, sort = False)

In [25]:
# Fill in missing values using appropriate data format for two core columns, NCESSCH and WEBTEXT:
full_data['NCESSCH'] = full_data['NCESSCH'].fillna(0)
full_data['WEBTEXT'] = full_data['WEBTEXT'].fillna("[]")

Make sure all the NCESSCH ids remain, aka make sure we kept the all the unique schools as before

In [27]:
li_ids = []
for n in set(original_df['NCESSCH']):
    if n not in set(full_data['NCESSCH']):
        print(n)
        li_ids.append(n)

In [28]:
len(set(li_ids))

0

In [ ]:
#Turn lists represented as strings into strings
full_data['WEBTEXT'] = full_data['WEBTEXT'].apply(ast.literal_eval)

In [32]:
original_df['WEBTEXT'] = original_df['WEBTEXT'].fillna(0)
original_df['MEMBER'] = original_df['MEMBER'].fillna(0)
original_df['AGE'] = original_df['AGE'].fillna(0)

In [33]:
original_df.drop_duplicates(original_df.columns.difference(['WEBTEXT', 'CMO_WEBTEXT']), keep = 'first', inplace = True)


In [34]:
original_df.reset_index(drop=True, inplace = True)

In [35]:
#check how unique NCESSCH school ids there are
orig_unique_ids = list(set(original_df['NCESSCH']))
len(orig_unique_ids)

10965

In [36]:
original_df

,CMO_NAME,CMO_MEMSUM,SCH_NAME,CMO_STATE,CMO_SCHNUM,CMO_URL,CMO_NUMSTATES,CMO_ALLSTATES,CMO_SECTOR,CMO_NUMSTUDENTS_CREDO17,...,STR,WEBTEXT_METHOD,ess_strength_y,prog_strength_y,PROG_COUNT,RIT_COUNT,ESS_COUNT,ESS_STR,PROG_STR,RIT_STR
0,AAEC,1320.0,ARIZONA AGRIBUSINESS & EQUINE CENTER - ESTRELLA,AZ,6.0,https://www.aaechighschools.com/,1.0,AZ,NaN,NaN,...,-480.000000,0,0.087500,0.037500,1,7,0,-6.000000,-2.977266,-2.132168
1,Arizona Agribusiness & Equine Center,1320.0,ARIZONA AGRIBUSINESS & EQUINE CENTER - ESTRELLA,AZ,6.0,https://www.aaechighschools.com/,1.0,AZ,CMO,1220.0,...,-480.000000,0,0.087500,0.037500,1,7,0,-6.000000,-2.977266,-2.132168
2,AAEC,1320.0,AAEC - SMCC CAMPUS,AZ,6.0,https://www.aaechighschools.com/,1.0,AZ,NaN,NaN,...,-434.000000,0,0.092505,0.097047,1,8,0,-6.000000,-2.941014,-2.037426
3,Arizona Agribusiness & Equine Center,1320.0,AAEC - SMCC CAMPUS,AZ,6.0,https://www.aaechighschools.com/,1.0,AZ,CMO,1220.0,...,-434.000000,0,0.092505,0.097047,1,8,0,-6.000000,-2.941014,-2.037426
4,AAEC,1320.0,AAEC - PARADISE VALLEY,AZ,6.0,https://www.aaechighschools.com/,1.0,AZ,NaN,NaN,...,-406.000000,0,0.083333,0.020833,0,1,0,-6.000000,-6.000000,-2.664642
5,Arizona Agribusiness & Equine Center,1320.0,AAEC - PARADISE VALLEY,AZ,6.0,https://www.aaechighschools.com/,1.0,AZ,CMO,1220.0,...,-406.000000,0,0.083333,0.020833,0,1,0,-6.000000,-6.000000,-2.664642
6,ACADEMY OF SKILLS AND KNOWLEDGE,1161.0,CUMBERLAND ACADEMY MIDDLE,TX,4.0,http://www.cumberlandacademy.com/,1.0,TX,NaN,NaN,...,12.469378,0,0.062500,0.234375,488,706,192,-2.547275,-2.141070,-1.981662
7,ACADEMY OF SKILLS AND KNOWLEDGE,1161.0,CUMBERLAND H S,TX,4.0,http://www.cumberlandacademy.com/,1.0,TX,NaN,NaN,...,NaN,0,NaN,NaN,37,140,28,-3.420362,-3.299189,-2.721245
8,ACADEMY OF SKILLS AND KNOWLEDGE,1161.0,CUMBERLAND ACADEMY,TX,4.0,http://www.cumberlandacademy.com/,1.0,TX,NaN,NaN,...,13.773116,0,0.062500,0.234375,582,826,13,-3.669989,-2.017585,-1.866895
9,ACCELERATED INTERMEDIATE ACADEMY,281.0,ACCELERATED INTERDISCIPLINARY ACAD,TX,3.0,http://www.aiacharterschools.org/,1.0,TX,NaN,NaN,...,16.676558,0,0.224490,0.081633,10,55,6,-2.871767,-2.649335,-1.909070


## Approach: 
We want to eventually compare the content of the webtext in the our full_data (overlap removed data) to the webtext in the original_df and for each unique NCESSCH id, we want to keep the more concise/cleaner webtext for that NCESSCH id. The more concise row will go into a new dataframe.

There's duplicate NCESSCH ids in the original_df and for each NCESSCH id group, we want to keep the row with most info.

In [38]:
#gets the row with the highest score = len(row['WEBTEXT']) + row['MEMBER'] + row['AGE']
def get_single_row(df):
    max_score = 0
    ind_max = 0
    r = df.iloc[0]
    print('next Group : ')
    for index, row in df.iterrows():
        #print('NCESSCH : ' + row['NCESSCH'] + "at index : " + index)        
        score = len(row['WEBTEXT']) + row['MEMBER'] + row['AGE']
        if score > max_score:
            ind_max = index
            r = row
            max_score = score
    #take the row with the highest score and add that to our new data frame
    print(ind_max)
    return r #return the max score row
    


In [39]:
#create the dataframe, where each row has a unique NCESSCH id and sufficient web text that
def create_unique(original_df):
    sub_df = original_df[['NCESSCH', 'WEBTEXT', 'MEMBER', 'AGE']]
    grouped = sub_df.groupby('NCESSCH')
    unique_df = pd.DataFrame(columns=['NCESSCH', 'WEBTEXT', 'MEMBER', 'AGE'])
    r = grouped.apply(get_single_row)
    unique_df = unique_df.append(r, ignore_index = True)
    return unique_df


Call the unique NCESSCH id df as unique_df


In [2]:
unique_df = create_unique(original_df)

In [48]:
len(unique_df['NCESSCH']) 

10965

In [49]:
len(set(unique_df['NCESSCH']))

10965

In [53]:
unique_df.reset_index(inplace = True)


In [58]:
#Sanity check: For the rows where the WEBTEXT entry is an empty list [], just look at original_df and make sure the row there also has a empty []
#Make sure the empty rows are correct

not_empty = []
for index, row in unique_df.iterrows():
    
    if (len(row['WEBTEXT']) == 0):
        #find that NCESSCH in the original_data and make sure it's empty as well
        a = original_df[original_df['NCESSCH'] == row['NCESSCH']]
        elem = a['WEBTEXT'].iloc[0] #use iloc for gettign row inside a for loop, normal [] indexing does not work for some reason
        if( len(elem) != 0 ):
            not_empty.append(row['NCESSCH'])
            print(row['NCESSCH'])

In [131]:
len(not_empty) #correct empty spots

0

Call the create_unique() function on full_data to keep the unique NCESSCH ids from the overlap removed data

In [3]:
full_unique_data = create_unique(full_data)    

In [64]:
len(full_unique_data['NCESSCH'])

10966

In [66]:
len(set(full_unique_data['NCESSCH']))

10966

In [69]:
full_unique_data = full_unique_data[full_unique_data.NCESSCH != 0]

In [70]:
len(full_unique_data['NCESSCH'])

10965

In [71]:
len(set(full_unique_data['NCESSCH']))

10965

In [76]:
li_nids = []
for n in list(unique_df['NCESSCH']):
    if n not in list(full_unique_data['NCESSCH']):
        print(n)
        li_nids.append(n)

In [77]:
len(li_nids)

0

full_unique_data (unique data from overlap removed data) and unique_df (unique data from original dataframe) have the same NCESSCH ids now. So, we can go ahead and compare the webtext of a row in full_unique_data to the webtext of the row with the corresponding NCESSCH id in unique_df and keep the shorter, and thus more "cleaner", version of the two 

In [ ]:
#Find the indices in full_unique_data for which we have to replace the with the content in original_d 
c = 0
li_replace_ind = []
li_nc = []
for index, row in full_unique_data.iterrows():
    orig_df = unique_df[unique_df['NCESSCH'] == row['NCESSCH']]
    orig_li_tup = orig_df['WEBTEXT'].iloc[0]

    num_orig = 0 #num characters
    for tup in orig_li_tup:
        if (len(tup) > 3):
            num_orig += len(tup[3])
    
    mer_li_tup = row['WEBTEXT']
    num_mer = 0
    for tup in mer_li_tup:
        if (len(tup) > 3):
            num_mer += len(tup[3])
    
    if (num_orig < num_mer): #if the original unique version has a smaller number of webpages, we have to 
        #change the WEBTEXT element in full_unique_data
        li_replace_ind.append(index)
        li_nc.append(row['NCESSCH'])
        #print("need to repalce at index : " + str(index))
        

In [ ]:
#Some debugging work below

In [115]:
len(li_replace_ind)

757

In [124]:
print(li_replace_ind[0])
print(li_nc[0])

16
20021000268.0


In [125]:
li_tup = full_unique_data['WEBTEXT'][li_replace_ind[0]]
num = 0
for tup in li_tup:
    num += len(tup[3])
print(num)

33263


In [126]:
ni_tup = unique_df[unique_df['NCESSCH'] == li_nc[0]]['WEBTEXT'].iloc[0]
num_2 = 0
for tup in ni_tup:
    num_2 += len(tup[3])
print(num_2)

28591


In [119]:
len(li_tup) #weird, parsed has more, just replace those indeces then


11

In [120]:
len(ni_tup)

6

In [123]:
len(li_nc)

757

In [122]:
len(li_replace)

1514

In [127]:
ind_nc_li = []
for i in np.arange(len(li_replace_ind)):
    ind_nc_li.append([li_replace_ind[i], li_nc[i]])

In [131]:
for sub_list in ind_nc_li:
    new_tup = unique_df[unique_df['NCESSCH'] == sub_list[1]]['WEBTEXT'].iloc[0]
    full_unique_data.at[sub_list[0], 'WEBTEXT'] = new_tup

In [136]:
li_tup = full_unique_data['WEBTEXT'][li_replace_ind[4]]
num = 0
for tup in li_tup:
    num += len(tup[3])
print(num)

15369


In [135]:
ni_tup = unique_df[unique_df['NCESSCH'] == li_nc[4]]['WEBTEXT'].iloc[0]
num_2 = 0
for tup in ni_tup:
    num_2 += len(tup[3])
print(num_2)

15369


In [137]:
full_unique_data 
#now is good, now just merge this with other meta data
#so take NCESSCH column get the row and append to a whole new data frame with the full_unique_data WEBTEXT column

,NCESSCH,WEBTEXT,MEMBER,AGE
1,1.001970e+10,"[(http://www.maef.net/, False, 0, Evening Acad...",NaN,-1.0
2,2.000010e+10,[],177.0,16.0
3,2.001500e+10,"[(https://www.kgbsd.org/ketchikancharter, Fals...",163.0,17.0
4,2.001500e+10,[(http://tongassschool.org/classrooms/3rd-and-...,183.0,12.0
5,2.001800e+10,"[(https://www.asdk12.org/aquarian, False, 0, S...",379.0,17.0
6,2.001800e+10,[],541.0,17.0
7,2.001800e+10,[],NaN,17.0
8,2.001800e+10,[],NaN,17.0
9,2.001800e+10,"[(http://winterberrycharterschool.com/, False,...",267.0,10.0
10,2.001800e+10,[(https://www.asdk12.org/aboutschools/eagleaca...,177.0,10.0


In [139]:
len(set(original_df['NCESSCH']))

10965

In [142]:
lis = []
for n in list(orignal_df['NCESSCH']):
    if n not in list(full_unique_data['NCESSCH']):
        print(n)
        lis.append(n)


In [143]:
len(lis)
#so full_unique data has all the unique NCESSCH ids that were detected in original_data

0

Two columns in original_df had the same name, so we delete 1 column

In [173]:
original_df[['Area (Land)']]

,Area (Land),Area (Land)
0,NaN,NaN
1,NaN,NaN
2,NaN,NaN
3,NaN,NaN
4,NaN,NaN
5,NaN,NaN
6,NaN,NaN
7,NaN,NaN
8,NaN,NaN
9,NaN,NaN


In [165]:
orig_u_df = original_df.loc[:, ~original_df.columns.duplicated()]

In [166]:
#Now only 1 Area (Land) column kept
orig_u_df[['Area (Land)']] 

,Area (Land)
0,NaN
1,NaN
2,NaN
3,NaN
4,NaN
5,NaN
6,NaN
7,NaN
8,NaN
9,NaN


Our final dataframe, which we will call parsed_df, should have the same column names, except duplicate ones, as the original_df

In [196]:
parsed_df = pd.DataFrame(columns = orig_u_df.columns)

In [198]:
len(set(list(parsed_df.columns))) #there is a duplicate column

399

In [199]:
cols_list = list(parsed_df.columns)

In [5]:
frequency = collections.Counter(cols_list)
frequency

In [172]:
len(frequency) #culprit 'Area (Land)': 2,

399

Add the most cut down rows to parsed_df

In [200]:
r = 0
row_num = []
for index, row in full_unique_data.iterrows():
    new_row = orig_u_df[orig_u_df['NCESSCH'] == row['NCESSCH']].head(1).reset_index() 
    #get full meta data for that row with specific NCESSCH
    li_tup = row['WEBTEXT']
#     print("row type : " + str(type(li_tup)))
#     print("orig_u type : " + str(type(new_row['WEBTEXT'].iloc[0])))
    new_row.at[0,'WEBTEXT'] = row['WEBTEXT'] #change the WEBTEXT column to be the parsed one in full_unique_data
    row_num.append(r)
    parsed_df = parsed_df.append(new_row, ignore_index = True)
    r+=1
    print(parsed_df.shape[0])
    
    

/opt/conda/lib/python3.6/site-packages/pandas/core/frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


1861
1862
1863
1864
1865
1866
1867
1868
1869
1870
1871
1872
1873
1874
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060


3500
3501
3502
3503
3504
3505
3506
3507
3508
3509
3510
3511
3512
3513
3514
3515
3516
3517
3518
3519
3520
3521
3522
3523
3524
3525
3526
3527
3528
3529
3530
3531
3532
3533
3534
3535
3536
3537
3538
3539
3540
3541
3542
3543
3544
3545
3546
3547
3548
3549
3550
3551
3552
3553
3554
3555
3556
3557
3558
3559
3560
3561
3562
3563
3564
3565
3566
3567
3568
3569
3570
3571
3572
3573
3574
3575
3576
3577
3578
3579
3580
3581
3582
3583
3584
3585
3586
3587
3588
3589
3590
3591
3592
3593
3594
3595
3596
3597
3598
3599
3600
3601
3602
3603
3604
3605
3606
3607
3608
3609
3610
3611
3612
3613
3614
3615
3616
3617
3618
3619
3620
3621
3622
3623
3624
3625
3626
3627
3628
3629
3630
3631
3632
3633
3634
3635
3636
3637
3638
3639
3640
3641
3642
3643
3644
3645
3646
3647
3648
3649
3650
3651
3652
3653
3654
3655
3656
3657
3658
3659
3660
3661
3662
3663
3664
3665
3666
3667
3668
3669
3670
3671
3672
3673
3674
3675
3676
3677
3678
3679
3680
3681
3682
3683
3684
3685
3686
3687
3688
3689
3690
3691
3692
3693
3694
3695
3696
3697
3698
3699


5139
5140
5141
5142
5143
5144
5145
5146
5147
5148
5149
5150
5151
5152
5153
5154
5155
5156
5157
5158
5159
5160
5161
5162
5163
5164
5165
5166
5167
5168
5169
5170
5171
5172
5173
5174
5175
5176
5177
5178
5179
5180
5181
5182
5183
5184
5185
5186
5187
5188
5189
5190
5191
5192
5193
5194
5195
5196
5197
5198
5199
5200
5201
5202
5203
5204
5205
5206
5207
5208
5209
5210
5211
5212
5213
5214
5215
5216
5217
5218
5219
5220
5221
5222
5223
5224
5225
5226
5227
5228
5229
5230
5231
5232
5233
5234
5235
5236
5237
5238
5239
5240
5241
5242
5243
5244
5245
5246
5247
5248
5249
5250
5251
5252
5253
5254
5255
5256
5257
5258
5259
5260
5261
5262
5263
5264
5265
5266
5267
5268
5269
5270
5271
5272
5273
5274
5275
5276
5277
5278
5279
5280
5281
5282
5283
5284
5285
5286
5287
5288
5289
5290
5291
5292
5293
5294
5295
5296
5297
5298
5299
5300
5301
5302
5303
5304
5305
5306
5307
5308
5309
5310
5311
5312
5313
5314
5315
5316
5317
5318
5319
5320
5321
5322
5323
5324
5325
5326
5327
5328
5329
5330
5331
5332
5333
5334
5335
5336
5337
5338


6779
6780
6781
6782
6783
6784
6785
6786
6787
6788
6789
6790
6791
6792
6793
6794
6795
6796
6797
6798
6799
6800
6801
6802
6803
6804
6805
6806
6807
6808
6809
6810
6811
6812
6813
6814
6815
6816
6817
6818
6819
6820
6821
6822
6823
6824
6825
6826
6827
6828
6829
6830
6831
6832
6833
6834
6835
6836
6837
6838
6839
6840
6841
6842
6843
6844
6845
6846
6847
6848
6849
6850
6851
6852
6853
6854
6855
6856
6857
6858
6859
6860
6861
6862
6863
6864
6865
6866
6867
6868
6869
6870
6871
6872
6873
6874
6875
6876
6877
6878
6879
6880
6881
6882
6883
6884
6885
6886
6887
6888
6889
6890
6891
6892
6893
6894
6895
6896
6897
6898
6899
6900
6901
6902
6903
6904
6905
6906
6907
6908
6909
6910
6911
6912
6913
6914
6915
6916
6917
6918
6919
6920
6921
6922
6923
6924
6925
6926
6927
6928
6929
6930
6931
6932
6933
6934
6935
6936
6937
6938
6939
6940
6941
6942
6943
6944
6945
6946
6947
6948
6949
6950
6951
6952
6953
6954
6955
6956
6957
6958
6959
6960
6961
6962
6963
6964
6965
6966
6967
6968
6969
6970
6971
6972
6973
6974
6975
6976
6977
6978


8418
8419
8420
8421
8422
8423
8424
8425
8426
8427
8428
8429
8430
8431
8432
8433
8434
8435
8436
8437
8438
8439
8440
8441
8442
8443
8444
8445
8446
8447
8448
8449
8450
8451
8452
8453
8454
8455
8456
8457
8458
8459
8460
8461
8462
8463
8464
8465
8466
8467
8468
8469
8470
8471
8472
8473
8474
8475
8476
8477
8478
8479
8480
8481
8482
8483
8484
8485
8486
8487
8488
8489
8490
8491
8492
8493
8494
8495
8496
8497
8498
8499
8500
8501
8502
8503
8504
8505
8506
8507
8508
8509
8510
8511
8512
8513
8514
8515
8516
8517
8518
8519
8520
8521
8522
8523
8524
8525
8526
8527
8528
8529
8530
8531
8532
8533
8534
8535
8536
8537
8538
8539
8540
8541
8542
8543
8544
8545
8546
8547
8548
8549
8550
8551
8552
8553
8554
8555
8556
8557
8558
8559
8560
8561
8562
8563
8564
8565
8566
8567
8568
8569
8570
8571
8572
8573
8574
8575
8576
8577
8578
8579
8580
8581
8582
8583
8584
8585
8586
8587
8588
8589
8590
8591
8592
8593
8594
8595
8596
8597
8598
8599
8600
8601
8602
8603
8604
8605
8606
8607
8608
8609
8610
8611
8612
8613
8614
8615
8616
8617


10048
10049
10050
10051
10052
10053
10054
10055
10056
10057
10058
10059
10060
10061
10062
10063
10064
10065
10066
10067
10068
10069
10070
10071
10072
10073
10074
10075
10076
10077
10078
10079
10080
10081
10082
10083
10084
10085
10086
10087
10088
10089
10090
10091
10092
10093
10094
10095
10096
10097
10098
10099
10100
10101
10102
10103
10104
10105
10106
10107
10108
10109
10110
10111
10112
10113
10114
10115
10116
10117
10118
10119
10120
10121
10122
10123
10124
10125
10126
10127
10128
10129
10130
10131
10132
10133
10134
10135
10136
10137
10138
10139
10140
10141
10142
10143
10144
10145
10146
10147
10148
10149
10150
10151
10152
10153
10154
10155
10156
10157
10158
10159
10160
10161
10162
10163
10164
10165
10166
10167
10168
10169
10170
10171
10172
10173
10174
10175
10176
10177
10178
10179
10180
10181
10182
10183
10184
10185
10186
10187
10188
10189
10190
10191
10192
10193
10194
10195
10196
10197
10198
10199
10200
10201
10202
10203
10204
10205
10206
10207
10208
10209
10210
10211
10212
10213
1021

In [202]:
parsed_df

,"% Civilian Population 16 to 19 Years: High School Graduate, or Enrolled (in School)","% Civilian Population 16 to 19 Years: Not High School Graduate, Not Enrolled (Dropped Out)",% Civilian Population in Labor Force 16 Years and Over: Employed,% Civilian Population in Labor Force 16 Years and Over: Unemployed,"% Employed Civilian Population 16 Years and Over: Agriculture, Forestry, Fishing and Hunting, and Mining","% Employed Civilian Population 16 Years and Over: Arts, Entertainment, and Recreation, and Accommodation and Food Services",% Employed Civilian Population 16 Years and Over: Construction,"% Employed Civilian Population 16 Years and Over: Educational Services, and Health Care and Social Assistance","% Employed Civilian Population 16 Years and Over: Finance and Insurance, and Real Estate and Rental and Leasing",% Employed Civilian Population 16 Years and Over: Information,...,close_rate13,close_rate14,close_rate15,close_rate16,close_rate99,ess_strength_x,ess_strength_y,index,prog_strength_x,prog_strength_y
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,0.000000,0.0000,0.000000,0.000000,NaN,NaN,11229.0,NaN,NaN
1,88.03,11.97,81.70,18.30,0.83,3.50,3.89,36.46,3.12,1.71,...,0.000000,0.000000,0.0000,0.000000,0.000000,0.137931,0.137931,6555.0,0.034483,0.034483
2,89.50,10.50,92.43,7.57,4.12,7.45,7.89,21.39,5.10,2.36,...,0.000000,0.000000,0.0000,0.100000,0.000000,0.041058,0.041058,6557.0,0.105839,0.105839
3,89.50,10.50,92.43,7.57,4.12,7.45,7.89,21.39,5.10,2.36,...,0.000000,0.000000,0.0000,0.100000,0.000000,0.037500,0.037500,6556.0,0.087500,0.087500
4,96.88,3.12,93.95,6.05,3.63,10.46,6.39,23.01,4.86,2.44,...,0.000000,0.000000,0.0000,0.000000,0.010000,0.095385,0.095385,6565.0,0.087179,0.087179
5,96.88,3.12,93.95,6.05,3.63,10.46,6.39,23.01,4.86,2.44,...,0.000000,0.000000,0.0000,0.000000,0.010000,0.114286,0.114286,6564.0,0.028571,0.028571
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,0.000000,0.0000,0.000000,0.010000,NaN,NaN,3754.0,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,0.000000,0.0000,0.000000,0.010000,NaN,NaN,3755.0,NaN,NaN
8,96.88,3.12,93.95,6.05,3.63,10.46,6.39,23.01,4.86,2.44,...,0.000000,0.000000,0.0000,0.000000,0.010000,0.040000,0.040000,6563.0,0.320000,0.320000
9,96.88,3.12,93.95,6.05,3.63,10.46,6.39,23.01,4.86,2.44,...,0.000000,0.000000,0.0000,0.000000,0.010000,0.299401,0.299401,6566.0,0.101796,0.101796


In [204]:
len((parsed_df['NCESSCH']))

10965

In [207]:
parsed_df['WEBTEXT'].isnull().sum()

0

In [208]:
parsed_df['NCESSCH'].isnull().sum()

0

parsed_df has final version of the cleaned overlap removed WEBTEXT data 

In [214]:
parsed_df.to_csv(folder_prefix+"nowdata/parsing/overlaps_removed_df.csv" , mode="w", index=False, header=parsed_df.columns.values, sep="\t", encoding="utf-8")

In [28]:
the_parsed.to_csv(folder_prefix+"nowdata/parsing/overlaps_removed_df.csv" , mode="w", index=False, header=the_parsed.columns.values, sep="\t", encoding="utf-8")

## We run a similar process for the data frame with now overlap removed CMO_WEBTEXT 

We first look at CMO_WEBTEXT_1.csv, for testing and preliminary investigation, since we were waiting form CMO_WEBTEXT_2.csv to finish. Once CMO_WEBTEXT_2.csv finished, we then followed the similar concatenation process as for the WEBTEXT column.

Note: The CMO_WEBTEXT_1 and CMO_WEBTEXT_2 dataframes do NOT have the overlap removed WEBTEXT in them.

In [19]:
cmo = pd.read_csv(folder_prefix + "nowdata/parsing/CMO_WEBTEXT_1.csv", sep="\t", low_memory=False, encoding="utf-8")

In [21]:
cmo

,"% Civilian Population 16 to 19 Years: High School Graduate, or Enrolled (in School)","% Civilian Population 16 to 19 Years: Not High School Graduate, Not Enrolled (Dropped Out)",% Civilian Population in Labor Force 16 Years and Over: Employed,% Civilian Population in Labor Force 16 Years and Over: Unemployed,"% Employed Civilian Population 16 Years and Over: Agriculture, Forestry, Fishing and Hunting, and Mining","% Employed Civilian Population 16 Years and Over: Arts, Entertainment, and Recreation, and Accommodation and Food Services",% Employed Civilian Population 16 Years and Over: Construction,"% Employed Civilian Population 16 Years and Over: Educational Services, and Health Care and Social Assistance","% Employed Civilian Population 16 Years and Over: Finance and Insurance, and Real Estate and Rental and Leasing",% Employed Civilian Population 16 Years and Over: Information,...,close_rate13,close_rate14,close_rate15,close_rate16,close_rate99,ess_strength_x,ess_strength_y,index,prog_strength_x,prog_strength_y
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.000000,0.0,0.0,0.000000,NaN,NaN,11229.0,NaN,NaN
1,88.03,11.97,81.70,18.30,0.83,3.50,3.89,36.46,3.12,1.71,...,0.0,0.000000,0.0,0.0,0.000000,0.137931,0.137931,6555.0,0.034483,0.034483
2,89.50,10.50,92.43,7.57,4.12,7.45,7.89,21.39,5.10,2.36,...,0.0,0.000000,0.0,0.1,0.000000,0.041058,0.041058,6557.0,0.105839,0.105839
3,89.50,10.50,92.43,7.57,4.12,7.45,7.89,21.39,5.10,2.36,...,0.0,0.000000,0.0,0.1,0.000000,0.037500,0.037500,6556.0,0.087500,0.087500
4,96.88,3.12,93.95,6.05,3.63,10.46,6.39,23.01,4.86,2.44,...,0.0,0.000000,0.0,0.0,0.010000,0.095385,0.095385,6565.0,0.087179,0.087179
5,96.88,3.12,93.95,6.05,3.63,10.46,6.39,23.01,4.86,2.44,...,0.0,0.000000,0.0,0.0,0.010000,0.114286,0.114286,6564.0,0.028571,0.028571
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.000000,0.0,0.0,0.010000,NaN,NaN,3754.0,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.000000,0.0,0.0,0.010000,NaN,NaN,3755.0,NaN,NaN
8,96.88,3.12,93.95,6.05,3.63,10.46,6.39,23.01,4.86,2.44,...,0.0,0.000000,0.0,0.0,0.010000,0.040000,0.040000,6563.0,0.320000,0.320000
9,96.88,3.12,93.95,6.05,3.63,10.46,6.39,23.01,4.86,2.44,...,0.0,0.000000,0.0,0.0,0.010000,0.299401,0.299401,6566.0,0.101796,0.101796


In [23]:
#see which NCESSCH are in the_parsed but not in cmo, and make a df
li_n = list(cmo['NCESSCH'])
left = the_parsed[~the_parsed['NCESSCH'].isin(li_n)]

In [25]:
left.shape[0] + cmo.shape[0] == the_parsed.shape[0]

True

In [29]:
left.to_csv(folder_prefix+"nowdata/parsing/cmo_left.csv" , mode="w", index=False, header=left.columns.values, sep="\t", encoding="utf-8")

In [7]:
ct = 0
t = []
for index, row in cmo.iterrows():
    if(len(row['CMO_WEBTEXT']) != 0):
        #unique_df = unique_df.append(row, ignore_index = True)
        t.append(row['NCESSCH'] )
 
    ct+=1
    print(ct)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


2296
2297
2298
2299
2300
2301
2302
2303
2304
2305
2306
2307
2308
2309
2310
2311
2312
2313
2314
2315
2316
2317
2318
2319
2320
2321
2322
2323
2324
2325
2326
2327
2328
2329
2330
2331
2332
2333
2334
2335
2336
2337
2338
2339
2340
2341
2342
2343
2344
2345
2346
2347
2348
2349
2350
2351
2352
2353
2354
2355
2356
2357
2358
2359
2360
2361
2362
2363
2364
2365
2366
2367
2368
2369
2370
2371
2372
2373
2374
2375
2376
2377
2378
2379
2380
2381
2382
2383
2384
2385
2386
2387
2388
2389
2390
2391
2392
2393
2394
2395
2396
2397
2398
2399
2400
2401
2402
2403
2404
2405
2406
2407
2408
2409
2410
2411
2412
2413
2414
2415
2416
2417
2418
2419
2420
2421
2422
2423
2424
2425
2426
2427
2428
2429
2430
2431
2432
2433
2434
2435
2436
2437
2438
2439
2440
2441
2442
2443
2444
2445
2446
2447
2448
2449
2450
2451
2452
2453
2454
2455
2456
2457
2458
2459
2460
2461
2462
2463
2464
2465
2466
2467
2468
2469
2470
2471
2472
2473
2474
2475
2476
2477
2478
2479
2480
2481
2482
2483
2484
2485
2486
2487
2488
2489
2490
2491
2492
2493
2494
2495


In [8]:
len(t)

4031

The_parsed has our overlap removed WEBTEXT data. We compare cmo df with parsed_df

In [5]:
the_parsed = pd.read_csv(folder_prefix + "nowdata/parsing/overlaps_removed_df.csv", sep="\t", low_memory=False, encoding="utf-8")

In [6]:
the_parsed['WEBTEXT'] =the_parsed['WEBTEXT'].apply(ast.literal_eval)

In [ ]:
the_parsed

In [ ]:
empty = []
#make sure the empty rows are correct
for index, row in the_parsed.iterrows():
    
    if (len(row['WEBTEXT']) == 0):
        #find that NCESSCH in the original_data and make sure it's empty as well
        empty.append(index)
        print(index)
    

In [12]:
the_parsed[the_parsed['CMO_WEBTEXT'].notnull()]['CMO_WEBTEXT']
#.sort_index(axis=1) #how to get alphabetically

,"% Civilian Population 16 to 19 Years: High School Graduate, or Enrolled (in School)","% Civilian Population 16 to 19 Years: Not High School Graduate, Not Enrolled (Dropped Out)",% Civilian Population in Labor Force 16 Years and Over: Employed,% Civilian Population in Labor Force 16 Years and Over: Unemployed,"% Employed Civilian Population 16 Years and Over: Agriculture, Forestry, Fishing and Hunting, and Mining","% Employed Civilian Population 16 Years and Over: Arts, Entertainment, and Recreation, and Accommodation and Food Services",% Employed Civilian Population 16 Years and Over: Construction,"% Employed Civilian Population 16 Years and Over: Educational Services, and Health Care and Social Assistance","% Employed Civilian Population 16 Years and Over: Finance and Insurance, and Real Estate and Rental and Leasing",% Employed Civilian Population 16 Years and Over: Information,...,close_rate13,close_rate14,close_rate15,close_rate16,close_rate99,ess_strength_x,ess_strength_y,index,prog_strength_x,prog_strength_y
39,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,0.000000,0.00000,0.000000,0.000000,0.090646,0.090646,649.0,0.203022,0.203022
40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,0.000000,0.00000,0.000000,0.000000,0.075949,0.075949,650.0,0.164557,0.164557
46,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,0.000000,0.00000,0.000000,0.000000,0.035714,0.035714,2008.0,0.330357,0.330357
57,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,0.000000,0.00000,0.000000,0.000000,NaN,NaN,472.0,NaN,NaN
58,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,0.000000,0.00000,0.000000,0.000000,NaN,NaN,473.0,NaN,NaN
95,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,0.000000,0.00000,0.000000,0.500000,NaN,NaN,1325.0,NaN,NaN
96,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,0.000000,0.00000,0.000000,0.500000,0.071429,0.071429,1323.0,0.357143,0.357143
122,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,0.000000,0.20000,0.000000,0.000000,NaN,NaN,469.0,NaN,NaN
123,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,0.000000,0.20000,0.000000,0.000000,NaN,NaN,470.0,NaN,NaN
124,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,0.000000,0.20000,0.000000,0.000000,NaN,NaN,471.0,NaN,NaN


In [25]:
the_parsed['CMO_WEBTEXT'] = the_parsed['CMO_WEBTEXT'].fillna("[]")
the_parsed['CMO_WEBTEXT'] = the_parsed['CMO_WEBTEXT'].apply(ast.literal_eval)

In [ ]:
original_df[original_df['CMO_WEBTEXT'].notnull()]['CMO_WEBTEXT']

See how many rows in CMO_WEBTEXT column have actual data in them.

Follow some debugging and testing of finding such rows


In [7]:
count=0
for index, row in the_parsed.iterrows():
    if(len(row['CMO_WEBTEXT']) >0):
        count+=1
count    

In [27]:
unique_df = pd.DataFrame(columns=original_df.columns)
lp = []

In [35]:
len(set(original_df['NCESSCH']))

10965

In [ ]:
#8432 rows in original_df (with repeats) have no CMO_WEBTEXT , []
#4445 have CMO_WEBTEXT (with repeats)
#2539 rows have actual CMO_WEBTEXt, 2544 close

In [ ]:
ct = 0
t = []
for index, row in original_df.iterrows():
    if(len(row['CMO_WEBTEXT']) != 0):
        #unique_df = unique_df.append(row, ignore_index = True)
        t.append(row['NCESSCH'] )
 
    ct+=1
    print(ct)
    

In [46]:
len(set(t))

2539

In [45]:
len(t)

4445

In [3]:
ct = 0
for index, row in original_df.iterrows():
    if(row['NCESSCH'] not in lp) and (len(row['CMO_WEBTEXT']) != 0:
        unique_df = unique_df.append(row, ignore_index = True)
        lp.append(row['NCESSCH'] )
 
    ct+=1
    print(ct)
    

In [36]:
len(lp)

10965

In [37]:
unique_df

,CMO_NAME,CMO_MEMSUM,SCH_NAME,CMO_STATE,CMO_SCHNUM,CMO_URL,CMO_NUMSTATES,CMO_ALLSTATES,CMO_SECTOR,CMO_NUMSTUDENTS_CREDO17,...,STR,WEBTEXT_METHOD,ess_strength_y,prog_strength_y,PROG_COUNT,RIT_COUNT,ESS_COUNT,ESS_STR,PROG_STR,RIT_STR
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,30.846154,0,NaN,NaN,223,346,38,-3.289894,-2.520768,-2.330003
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,17.364398,0,0.165213,0.086469,5,33,4,-3.854746,-3.757806,-2.938141
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,18.720930,0,0.109244,0.227941,20,87,13,-2.742846,-2.554973,-1.916847
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,30.366972,0,0.128060,0.098870,6,12,13,-2.740545,-3.075973,-2.775307
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,23.458833,0,NaN,NaN,62,283,209,-2.633367,-3.161696,-2.502360
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,32.200000,0,0.102025,0.234620,87,276,81,-2.448344,-2.416373,-1.915480
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-222.000000,0,0.076531,0.335459,1,16,0,-6.000000,-4.106089,-2.902003
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,23.100000,0,0.134615,0.134615,124,239,34,-2.734400,-2.169930,-1.885806
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,47.363636,0,0.054662,0.113869,162,110,30,-3.272978,-2.540422,-2.708629
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,50.000000,0,0.144578,0.036145,36,265,36,-3.020373,-3.020131,-2.153119


In [2]:
the_parsed[the_parsed['WEBTEXT'].notnull()]

We're using nowdata instead of original_df data as our "original data"

We don't have WEBTEXT cleaned in curr_df (in charters_2015.pkl)

In [7]:

curr_df = pd.read_pickle(folder_prefix+ 'nowdata/charters_2015.pkl')

In [8]:
curr_df

,NCESSCH,URL,LAT1516,LON1516,AM,AS,BL,HI,HP,TR,...,ESS_VALID_STR,PROG_VALID_COUNT,PROG_VALID_STR,RIT_VALID_COUNT,RIT_VALID_STR,INQUIRY_RATIO,DISCIPLINE_RATIO,ESS_VALID_RATIO,PROG_VALID_RATIO,RIT_VALID_RATIO
0,1.001970e+10,http://www.maef.net/,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-3.049363,18.0,-2.997944,51.0,-2.545622,0.007424,0.000446,0.000893,0.001005,0.002847
1,2.000010e+10,https://education.alaska.gov/DOE_Rolodex/Schoo...,60.796131,-161.765194,167.0,0.0,0.0,0.0,0.0,0.0,...,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
2,2.001500e+10,https://www.kgbsd.org/ketchikancharter,55.347001,-131.641191,74.0,37.0,2.0,5.0,4.0,5.0,...,-6.000000,1.0,-3.123525,0.0,-6.000000,0.003762,0.000752,0.000000,0.000752,0.000000
3,2.001500e+10,http://www.tongassschool.org/,55.347001,-131.641191,57.0,12.0,4.0,6.0,1.0,11.0,...,-3.673297,5.0,-3.752448,31.0,-2.960010,0.009768,0.000177,0.000212,0.000177,0.001096
4,2.001800e+10,https://aquarian.asdk12.org/,61.192407,-149.916872,10.0,11.0,6.0,19.0,2.0,51.0,...,-6.000000,0.0,-6.000000,0.0,-6.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,2.001800e+10,https://education.alaska.gov/DOE_Rolodex/Schoo...,61.198100,-149.876000,30.0,16.0,18.0,43.0,18.0,69.0,...,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
6,2.001800e+10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
7,2.001800e+10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
8,2.001800e+10,http://www.winterberrycharterschool.com/,61.194450,-149.791641,15.0,8.0,5.0,14.0,1.0,26.0,...,-3.492173,10.0,-3.190948,34.0,-2.659441,0.009728,0.000193,0.000322,0.000644,0.002191
9,2.001800e+10,http://www.asdk12.org/aboutschools/eagleacademy/,61.319213,-149.579442,2.0,3.0,8.0,6.0,0.0,18.0,...,-6.000000,0.0,-6.000000,0.0,-6.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [9]:
cmo_1_df = pd.read_csv(folder_prefix + "nowdata/parsing/CMO_WEBTEXT_1.csv", sep="\t", low_memory=False, encoding="utf-8")

cmo_1_df and cmo_2_df are the two dataframes with overlap removed CMO_WEBTEXT

In [11]:
cmo_2_df = pd.read_csv(folder_prefix + "nowdata/parsing/CMO_WEBTEXT_2.csv", sep="\t", low_memory=False, encoding="utf-8")

In [34]:
#Turn string of lists into lists
cmo_1_df['CMO_WEBTEXT'] = cmo_1_df['CMO_WEBTEXT'].apply(ast.literal_eval)
cmo_2_df['CMO_WEBTEXT'] = cmo_2_df['CMO_WEBTEXT'].apply(ast.literal_eval)


In [35]:
for i in range(cmo_1_df.shape[0]):
    if len(cmo_1_df['CMO_WEBTEXT'][0]) != 0:
        print(i)

In [38]:
len(cmo_1_df['CMO_WEBTEXT'][0])

0

In [44]:
cmo_1_df['CMO_WEBTEXT'][39]

[('https://casschools.com/',
  'False',
  '0',
  '\t\t\t\t\tCenter for Academic Success:  K-12\t\t\t\t\nHigher Standards.  Higher Expectations.\nMenu\nHome\nAbout CAS\nFAQs\nSchool Calendar 2018-19\nBreakfast & Lunch Menus\nSchool Wellness Policy\nParentâ\x80\x99s Right to Know-Teacher Qulifications\nProposed Budget FY 2019\nBudget Adoption Hearing Notice\nSierra Vista Campus\nSV Elementary School\nSV Middle School\nSV High School\nA+ Anywhere Login\nSV Administration\nDouglas Campus\nDouglas Elementary School\nDouglas Middle School\nDouglas High School\nDouglas Administration\nBoard of Directors\nForms & Resources\nParent/Student Handbook â\x80\x93 Elementary/Middle School\nParent/Student Handbook â\x80\x93 High School\nTranscript / Diploma Request\nNew Student Applications\nReturning Student Application\nPowerSchool Login\nTax Credit 2017\nBefore & After Care Clubs\nHigh School Club Schedule\nCyberPatriots\nSolar Go-Kart Team\nEmployment\nApply Now\nContact Us\n\t\t\t\tHigher Standar

In [ ]:
#Find the rows that are not empty in CMO_WEBTEXT
c = 0
for w in cmo_1_df['CMO_WEBTEXT']:
    if len(w) != 0:
        print("ind : " + str(c) + " with length " + str(len(w)))
    c+=1

Concatenate cmo dataframes into full_cmo_data

In [28]:
full_cmo_data = pd.concat([cmo_1_df, cmo_2_df], ignore_index = True, sort = False)

In [29]:
full_cmo_data.shape[0]

10965

In [30]:
full_cmo_data['CMO_WEBTEXT'] = full_cmo_data['CMO_WEBTEXT'].apply(ast.literal_eval)


In [ ]:
#Find the rows that are not empty in CMO_WEBTEXT column
cmo_web = full_cmo_data['CMO_WEBTEXT']
for i in range(full_cmo_data.shape[0]):
    if len(full_cmo_data['CMO_WEBTEXT'][i]) != 0:
        print(i)

In [19]:
li_ids = []
for n in list(curr_df['NCESSCH']):
    if n not in list(full_cmo_data['NCESSCH']):
        print(n)
        li_ids.append(n)

In [20]:
#curr_df and full_cmo_data have same exact NCESSCH ids
li_ids

[]

Now find row in curr_df where NCESSCH id matches in full_cmo_data and overwrite the CMO_WEBTEXT in curr_data with the one in full_cmo_data

In [58]:
for index, row in curr_df.iterrows():
        #print('NCESSCH : ' + row['NCESSCH'] + "at index : " + index)
        cmo_li_tuples = full_cmo_data[full_cmo_data['NCESSCH'] == row['NCESSCH']]['CMO_WEBTEXT'].iloc[0]
        curr_df.at[index, 'CMO_WEBTEXT'] = cmo_li_tuples

In [60]:
curr_df.iloc[39]['CMO_WEBTEXT']

[('https://casschools.com/',
  'False',
  '0',
  '\t\t\t\t\tCenter for Academic Success:  K-12\t\t\t\t\nHigher Standards.  Higher Expectations.\nMenu\nHome\nAbout CAS\nFAQs\nSchool Calendar 2018-19\nBreakfast & Lunch Menus\nSchool Wellness Policy\nParentâ\x80\x99s Right to Know-Teacher Qulifications\nProposed Budget FY 2019\nBudget Adoption Hearing Notice\nSierra Vista Campus\nSV Elementary School\nSV Middle School\nSV High School\nA+ Anywhere Login\nSV Administration\nDouglas Campus\nDouglas Elementary School\nDouglas Middle School\nDouglas High School\nDouglas Administration\nBoard of Directors\nForms & Resources\nParent/Student Handbook â\x80\x93 Elementary/Middle School\nParent/Student Handbook â\x80\x93 High School\nTranscript / Diploma Request\nNew Student Applications\nReturning Student Application\nPowerSchool Login\nTax Credit 2017\nBefore & After Care Clubs\nHigh School Club Schedule\nCyberPatriots\nSolar Go-Kart Team\nEmployment\nApply Now\nContact Us\n\t\t\t\tHigher Standar

In [61]:
#make sure each cmo_webtext in curr_df has been replaced correctly
for index, row in curr_df.iterrows(): 
        #print('NCESSCH : ' + row['NCESSCH'] + "at index : " + index)
        cmo_li_tuples = full_cmo_data[full_cmo_data['NCESSCH'] == row['NCESSCH']]['CMO_WEBTEXT'].iloc[0]
        if(row['CMO_WEBTEXT'] != cmo_li_tuples):
            print(index)
        


curr_df does not have overlap removed WEBTEXT data in it

In [62]:
curr_df.to_pickle(folder_prefix+"nowdata/charters_final_2015.pkl") 

In [63]:
a = pd.read_pickle(folder_prefix+"nowdata/charters_final_2015.pkl")

In [64]:
a

,NCESSCH,URL,LAT1516,LON1516,AM,AS,BL,HI,HP,TR,...,ESS_VALID_STR,PROG_VALID_COUNT,PROG_VALID_STR,RIT_VALID_COUNT,RIT_VALID_STR,INQUIRY_RATIO,DISCIPLINE_RATIO,ESS_VALID_RATIO,PROG_VALID_RATIO,RIT_VALID_RATIO
0,1.001970e+10,http://www.maef.net/,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-3.049363,18.0,-2.997944,51.0,-2.545622,0.007424,0.000446,0.000893,0.001005,0.002847
1,2.000010e+10,https://education.alaska.gov/DOE_Rolodex/Schoo...,60.796131,-161.765194,167.0,0.0,0.0,0.0,0.0,0.0,...,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
2,2.001500e+10,https://www.kgbsd.org/ketchikancharter,55.347001,-131.641191,74.0,37.0,2.0,5.0,4.0,5.0,...,-6.000000,1.0,-3.123525,0.0,-6.000000,0.003762,0.000752,0.000000,0.000752,0.000000
3,2.001500e+10,http://www.tongassschool.org/,55.347001,-131.641191,57.0,12.0,4.0,6.0,1.0,11.0,...,-3.673297,5.0,-3.752448,31.0,-2.960010,0.009768,0.000177,0.000212,0.000177,0.001096
4,2.001800e+10,https://aquarian.asdk12.org/,61.192407,-149.916872,10.0,11.0,6.0,19.0,2.0,51.0,...,-6.000000,0.0,-6.000000,0.0,-6.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,2.001800e+10,https://education.alaska.gov/DOE_Rolodex/Schoo...,61.198100,-149.876000,30.0,16.0,18.0,43.0,18.0,69.0,...,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
6,2.001800e+10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
7,2.001800e+10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
8,2.001800e+10,http://www.winterberrycharterschool.com/,61.194450,-149.791641,15.0,8.0,5.0,14.0,1.0,26.0,...,-3.492173,10.0,-3.190948,34.0,-2.659441,0.009728,0.000193,0.000322,0.000644,0.002191
9,2.001800e+10,http://www.asdk12.org/aboutschools/eagleacademy/,61.319213,-149.579442,2.0,3.0,8.0,6.0,0.0,18.0,...,-6.000000,0.0,-6.000000,0.0,-6.000000,0.000000,0.000000,0.000000,0.000000,0.000000
